# Semantic Segmentation

In [3]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import time

print("All loaded.")

All loaded.


In [4]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), \
'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [5]:
# Download VGG model that is pretrained.
helper.maybe_download_pretrained_vgg('./data')

Done.


In [6]:
# Load pretrained model and test.
def load_vgg(sess, vgg_path):
    tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)
    print("Model loaded.")
    input_tensor = sess.graph.get_tensor_by_name('image_input:0')
    keep_prob_tensor = sess.graph.get_tensor_by_name('keep_prob:0')
    layer3_out_tensor = sess.graph.get_tensor_by_name('layer3_out:0')
    layer4_out_tensor = sess.graph.get_tensor_by_name('layer4_out:0')
    layer7_out_tensor = sess.graph.get_tensor_by_name('layer7_out:0')
    
    return input_tensor, keep_prob_tensor, layer3_out_tensor, layer4_out_tensor, layer7_out_tensor

print("Start test.")
tests.test_load_vgg(load_vgg, tf)

Start test.
Model loaded.
Tests Passed


In [7]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # Perserve spatial information with 1x1 convolution.
    conv_1_1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    # Decode and add the skip layer in.
    layer_decode_1 = tf.layers.conv2d_transpose(conv_1_1, num_classes, 4, strides=(2,2), padding='same',
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    vgg_layer4_in = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same',
                                     kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    add_layer_1 = tf.add(layer_decode_1, vgg_layer4_in)

    # Decode and add the skip layer in.
    layer_decode_2 = tf.layers.conv2d_transpose(add_layer_1, num_classes, 4, strides=(2,2), padding='same',
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    vgg_layer3_in = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same',
                                     kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    add_layer_2 = tf.add(layer_decode_2, vgg_layer3_in)

    # Bring back up to final size.
    layer_decode_3 = tf.layers.conv2d_transpose(add_layer_2, num_classes, 16, strides=(8,8), padding='same',
                                              kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    return layer_decode_3
tests.test_layers(layers)

Tests Passed


In [8]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)

Tests Passed


In [9]:
print("Hello")

Hello


In [10]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    print("In train nn")
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        i = 0
        total_loss = 0
        for image, label in get_batches_fn(batch_size):
            #print(len(image), len(label))
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict={
                input_image: image,
                correct_label: label,
                keep_prob: 0.75,
                learning_rate: 0.0005
                })
            #print('Batch ', i, 'Size', len(label), 'Loss ',loss)
            i += 1
            total_loss += loss
        print('Epoch ',epoch, 'Avg Loss ',(total_loss/max(i,1)))
    pass
            
print("Start test")
tests.test_train_nn(train_nn)
print("End test")

Start test
In train nn
Epoch  0 Avg Loss  10.1099996567
Tests Passed
End test


In [11]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    epochs = 100
    batch_size = 4
    
    with tf.Session() as sess:
        # Place holders.
        correct_label = tf.placeholder(tf.float32, shape=[None, image_shape[0],image_shape[1], num_classes])
        learning_rate = tf.placeholder(tf.float32, shape=[])
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        load_info = load_vgg(sess, vgg_path)
        last_layer = layers(load_info[2], load_info[3], load_info[4], num_classes)
        logit, train_op, cel = optimize(last_layer, correct_label, learning_rate, num_classes)
        
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cel, load_info[0],
                correct_label, load_info[1], learning_rate)
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logit, load_info[1], load_info[0])

        # OPTIONAL: Apply the trained model to a video
print("Done.")

Done.


In [12]:
# Run everything.
start = time.time()
run()
end = time.time()
print("Run time (min)", round((end - start)/60,1))

Tests Passed
Done.
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Model loaded.
In train nn
Epoch  0 Avg Loss  7.05092385207
Epoch  1 Avg Loss  0.585084765741
Epoch  2 Avg Loss  0.494509852096
Epoch  3 Avg Loss  0.389105883363
Epoch  4 Avg Loss  0.276134982297
Epoch  5 Avg Loss  0.228826101512
Epoch  6 Avg Loss  0.209771111812
Epoch  7 Avg Loss  0.186986239601
Epoch  8 Avg Loss  0.175348541712
Epoch  9 Avg Loss  0.168495912972
Epoch  10 Avg Loss  0.15671658669
Epoch  11 Avg Loss  0.148937277088
Epoch  12 Avg Loss  0.13275846992
Epoch  13 Avg Loss  0.125813544511
Epoch  14 Avg Loss  0.112022119332
Epoch  15 Avg Loss  0.10355681033
Epoch  16 Avg Loss  0.0969984062935
Epoch  17 Avg Loss  0.0917572968014
Epoch  18 Avg Loss  0.0910545676845
Epoch  19 Avg Loss  0.0967383749069
Epoch  20 Avg Loss  0.0872483688266
Epoch  21 Avg Loss  0.0762031148762
Epoch  22 Avg Loss  0.0719759616337
Epoch  23 Avg Loss  0.0677068012524
Epoch  24 Avg Loss  0.063055678998
Epoch  25 

LR 0.0005, batch size 4.

    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  3.90204914792
    Epoch  1 Avg Loss  0.657484468532
    Epoch  2 Avg Loss  0.614491132024
    Epoch  3 Avg Loss  0.54804581567
    Epoch  4 Avg Loss  0.437992954907
    Epoch  5 Avg Loss  0.292067604318
    Epoch  6 Avg Loss  0.205530358504
    Epoch  7 Avg Loss  0.167591966689
    Epoch  8 Avg Loss  0.157602091562
    Epoch  9 Avg Loss  0.144370430238
    Training Finished. Saving test images to: ./runs/1504464437.7355258
    
    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  2.88289633516
    Epoch  1 Avg Loss  0.638864358811
    Epoch  2 Avg Loss  0.534451405071
    Epoch  3 Avg Loss  0.307159428319
    Epoch  4 Avg Loss  0.227333909028
    Epoch  5 Avg Loss  0.183531538571
    Epoch  6 Avg Loss  0.157775025866
    Epoch  7 Avg Loss  0.138731538433
    Epoch  8 Avg Loss  0.126251972395
    Epoch  9 Avg Loss  0.118869616357
    Epoch  10 Avg Loss  0.107102122209
    Epoch  11 Avg Loss  0.101455739355
    Epoch  12 Avg Loss  0.094617414403
    Epoch  13 Avg Loss  0.0900503712362
    Epoch  14 Avg Loss  0.0845060919042
    Epoch  15 Avg Loss  0.080587420485
    Epoch  16 Avg Loss  0.0690619717926
    Epoch  17 Avg Loss  0.0704193497459
    Epoch  18 Avg Loss  0.0760340576303
    Epoch  19 Avg Loss  0.066151058582
    Training Finished. Saving test images to: ./runs/1504465934.256854
    Run time (min) 14.3
    
    
    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  4.16452528189
    Epoch  1 Avg Loss  0.644734376914
    Epoch  2 Avg Loss  0.598660364543
    Epoch  3 Avg Loss  0.542042464018
    Epoch  4 Avg Loss  0.43155649386
    Epoch  5 Avg Loss  0.296896670167
    Epoch  6 Avg Loss  0.232903065543
    Epoch  7 Avg Loss  0.203807064932
    Epoch  8 Avg Loss  0.182478959226
    Epoch  9 Avg Loss  0.165944919092
    Epoch  10 Avg Loss  0.153618938609
    Epoch  11 Avg Loss  0.144780303619
    Epoch  12 Avg Loss  0.141919795781
    Epoch  13 Avg Loss  0.129554213216
    Epoch  14 Avg Loss  0.119814856208
    Epoch  15 Avg Loss  0.113332880686
    Epoch  16 Avg Loss  0.100943960249
    Epoch  17 Avg Loss  0.101050681549
    Epoch  18 Avg Loss  0.0932551691283
    Epoch  19 Avg Loss  0.088199434113
    Epoch  20 Avg Loss  0.0779577135938
    Epoch  21 Avg Loss  0.0749880286725
    Epoch  22 Avg Loss  0.102781322568
    Epoch  23 Avg Loss  0.0747446419339
    Epoch  24 Avg Loss  0.0633883358885
    Epoch  25 Avg Loss  0.0647972321316
    Epoch  26 Avg Loss  0.0695452677622
    Epoch  27 Avg Loss  0.0515487848794
    Epoch  28 Avg Loss  0.0473341953826
    Epoch  29 Avg Loss  0.0436629046214
    Epoch  30 Avg Loss  0.0437894125967
    Epoch  31 Avg Loss  0.0412249287769
    Epoch  32 Avg Loss  0.0371146716739
    Epoch  33 Avg Loss  0.0350831775122
    Epoch  34 Avg Loss  0.0338186220304
    Epoch  35 Avg Loss  0.0330502738385
    Epoch  36 Avg Loss  0.0317115500722
    Epoch  37 Avg Loss  0.0301372975698
    Epoch  38 Avg Loss  0.0297145159941
    Epoch  39 Avg Loss  0.0286542716347
    Epoch  40 Avg Loss  0.0275194326747
    Epoch  41 Avg Loss  0.0263443496078
    Epoch  42 Avg Loss  0.0256923375115
    Epoch  43 Avg Loss  0.0256232973826
    Epoch  44 Avg Loss  0.0257169005448
    Epoch  45 Avg Loss  0.0251738558817
    Epoch  46 Avg Loss  0.023793536519
    Epoch  47 Avg Loss  0.0235525638962
    Epoch  48 Avg Loss  0.0225480439728
    Epoch  49 Avg Loss  0.0222966195633
    Training Finished. Saving test images to: ./runs/1504480182.2509675
    Run time (min) 34.0
    
    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  7.05092385207
    Epoch  1 Avg Loss  0.585084765741
    Epoch  2 Avg Loss  0.494509852096
    Epoch  3 Avg Loss  0.389105883363
    Epoch  4 Avg Loss  0.276134982297
    Epoch  5 Avg Loss  0.228826101512
    Epoch  6 Avg Loss  0.209771111812
    Epoch  7 Avg Loss  0.186986239601
    Epoch  8 Avg Loss  0.175348541712
    Epoch  9 Avg Loss  0.168495912972
    Epoch  10 Avg Loss  0.15671658669
    Epoch  11 Avg Loss  0.148937277088
    Epoch  12 Avg Loss  0.13275846992
    Epoch  13 Avg Loss  0.125813544511
    Epoch  14 Avg Loss  0.112022119332
    Epoch  15 Avg Loss  0.10355681033
    Epoch  16 Avg Loss  0.0969984062935
    Epoch  17 Avg Loss  0.0917572968014
    Epoch  18 Avg Loss  0.0910545676845
    Epoch  19 Avg Loss  0.0967383749069
    Epoch  20 Avg Loss  0.0872483688266
    Epoch  21 Avg Loss  0.0762031148762
    Epoch  22 Avg Loss  0.0719759616337
    Epoch  23 Avg Loss  0.0677068012524
    Epoch  24 Avg Loss  0.063055678998
    Epoch  25 Avg Loss  0.059120658683
    Epoch  26 Avg Loss  0.0569193540779
    Epoch  27 Avg Loss  0.151632049006
    Epoch  28 Avg Loss  0.127475805274
    Epoch  29 Avg Loss  0.101657415957
    Epoch  30 Avg Loss  0.0917611541609
    Epoch  31 Avg Loss  0.0836557352482
    Epoch  32 Avg Loss  0.0686163976278
    Epoch  33 Avg Loss  0.0628029653063
    Epoch  34 Avg Loss  0.0577556702168
    Epoch  35 Avg Loss  0.0528224076412
    Epoch  36 Avg Loss  0.0539136618784
    Epoch  37 Avg Loss  0.0735539709766
    Epoch  38 Avg Loss  0.0562805585461
    Epoch  39 Avg Loss  0.0521814148771
    Epoch  40 Avg Loss  0.044860977705
    Epoch  41 Avg Loss  0.0430140760919
    Epoch  42 Avg Loss  0.0413095708426
    Epoch  43 Avg Loss  0.039371221288
    Epoch  44 Avg Loss  0.0380210633045
    Epoch  45 Avg Loss  0.0373553492078
    Epoch  46 Avg Loss  0.0360720809294
    Epoch  47 Avg Loss  0.0344820742125
    Epoch  48 Avg Loss  0.0389312170829
    Epoch  49 Avg Loss  0.0957680188426
    Epoch  50 Avg Loss  0.0592928073076
    Epoch  51 Avg Loss  0.0545642200653
    Epoch  52 Avg Loss  0.0417554730371
    Epoch  53 Avg Loss  0.0382771477205
    Epoch  54 Avg Loss  0.0351308450001
    Epoch  55 Avg Loss  0.033292067671
    Epoch  56 Avg Loss  0.0322292529506
    Epoch  57 Avg Loss  0.0311054635589
    Epoch  58 Avg Loss  0.030622759145
    Epoch  59 Avg Loss  0.0294077591078
    Epoch  60 Avg Loss  0.0279784048333
    Epoch  61 Avg Loss  0.0276837671584
    Epoch  62 Avg Loss  0.0271303190607
    Epoch  63 Avg Loss  0.0264956197918
    Epoch  64 Avg Loss  0.0252810238975
    Epoch  65 Avg Loss  0.025124378783
    Epoch  66 Avg Loss  0.0250398924013
    Epoch  67 Avg Loss  0.023706991991
    Epoch  68 Avg Loss  0.0233785144423
    Epoch  69 Avg Loss  0.0230589292637
    Epoch  70 Avg Loss  0.0228656323065
    Epoch  71 Avg Loss  0.0230590761831
    Epoch  72 Avg Loss  0.0226193914853
    Epoch  73 Avg Loss  0.0224087258342
    Epoch  74 Avg Loss  0.0232784351819
    Epoch  75 Avg Loss  0.0224100939343
    Epoch  76 Avg Loss  0.0209102803101
    Epoch  77 Avg Loss  0.0203999962071
    Epoch  78 Avg Loss  0.0199348941726
    Epoch  79 Avg Loss  0.0197321516781
    Epoch  80 Avg Loss  0.0200159012736
    Epoch  81 Avg Loss  0.019917894984
    Epoch  82 Avg Loss  0.0190431131688
    Epoch  83 Avg Loss  0.0193480357961
    Epoch  84 Avg Loss  0.0190306539737
    Epoch  85 Avg Loss  0.0185776990137
    Epoch  86 Avg Loss  0.0181874405235
    Epoch  87 Avg Loss  0.0186035396812
    Epoch  88 Avg Loss  0.0181440004364
    Epoch  89 Avg Loss  0.0189843688875
    Epoch  90 Avg Loss  0.026134693319
    Epoch  91 Avg Loss  0.149794096612
    Epoch  92 Avg Loss  0.0852214462851
    Epoch  93 Avg Loss  0.0793622156529
    Epoch  94 Avg Loss  0.0560250628663
    Epoch  95 Avg Loss  0.0401060819013
    Epoch  96 Avg Loss  0.0334997094964
    Epoch  97 Avg Loss  0.0278090135233
    Epoch  98 Avg Loss  0.0252614994087
    Epoch  99 Avg Loss  0.0224723077622
    Training Finished. Saving test images to: ./runs/1504484884.0345714
    Run time (min) 67.4

LR = 0.0005, batch size = 8

    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  6.41729480673
    Epoch  1 Avg Loss  0.707358105763
    Epoch  2 Avg Loss  0.674838037104
    Epoch  3 Avg Loss  0.654057364206
    Epoch  4 Avg Loss  0.632316102853
    Epoch  5 Avg Loss  0.602009723315
    Epoch  6 Avg Loss  0.559047156089
    Epoch  7 Avg Loss  0.482389046534
    Epoch  8 Avg Loss  0.369063418459
    Epoch  9 Avg Loss  0.273533546039
    Epoch  10 Avg Loss  0.238366590158
    Epoch  11 Avg Loss  0.204476247365
    Epoch  12 Avg Loss  0.191412621656
    Epoch  13 Avg Loss  0.175724354749
    Epoch  14 Avg Loss  0.157652310624
    Epoch  15 Avg Loss  0.148710209574
    Epoch  16 Avg Loss  0.139177301848
    Epoch  17 Avg Loss  0.133903230163
    Epoch  18 Avg Loss  0.125572590812
    Epoch  19 Avg Loss  0.118632158315
    Epoch  20 Avg Loss  0.107716327583
    Epoch  21 Avg Loss  0.101701840154
    Epoch  22 Avg Loss  0.101422866051
    Epoch  23 Avg Loss  0.118933733049
    Epoch  24 Avg Loss  0.128011132817
    Epoch  25 Avg Loss  0.0970143987923
    Epoch  26 Avg Loss  0.0908436194264
    Epoch  27 Avg Loss  0.0791686918284
    Epoch  28 Avg Loss  0.0782184527331
    Epoch  29 Avg Loss  0.0829253946809
    Training Finished. Saving test images to: ./runs/1504475618.883995
    Run time (min) 16.6
    
    Tests Passed
    
    
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  6.0487233352
    Epoch  1 Avg Loss  0.715834872143
    Epoch  2 Avg Loss  0.68106680303
    Epoch  3 Avg Loss  0.663457354984
    Epoch  4 Avg Loss  0.648617494751
    Epoch  5 Avg Loss  0.633478098624
    Epoch  6 Avg Loss  0.615973143964
    Epoch  7 Avg Loss  0.591240699227
    Epoch  8 Avg Loss  0.548376977444
    Epoch  9 Avg Loss  0.453881887165
    Epoch  10 Avg Loss  0.319732387324
    Epoch  11 Avg Loss  0.236770768423
    Epoch  12 Avg Loss  0.213131683904
    Epoch  13 Avg Loss  0.208642183526
    Epoch  14 Avg Loss  0.191654943534
    Epoch  15 Avg Loss  0.176294851343
    Epoch  16 Avg Loss  0.162667229772
    Epoch  17 Avg Loss  0.154086795006
    Epoch  18 Avg Loss  0.140694947259
    Epoch  19 Avg Loss  0.130278059156
    Epoch  20 Avg Loss  0.121115975283
    Epoch  21 Avg Loss  0.121860261503
    Epoch  22 Avg Loss  0.124357122827
    Epoch  23 Avg Loss  0.109607583768
    Epoch  24 Avg Loss  0.0990818124768
    Epoch  25 Avg Loss  0.0950042429204
    Epoch  26 Avg Loss  0.0884539146681
    Epoch  27 Avg Loss  0.0907491187792
    Epoch  28 Avg Loss  0.0935376060976
    Epoch  29 Avg Loss  0.0948246373115
    Epoch  30 Avg Loss  0.0798457080649
    Epoch  31 Avg Loss  0.0724696796891
    Epoch  32 Avg Loss  0.0692710892574
    Epoch  33 Avg Loss  0.0706085158965
    Epoch  34 Avg Loss  0.0662733336558
    Epoch  35 Avg Loss  0.0597398362651
    Epoch  36 Avg Loss  0.056767708345
    Epoch  37 Avg Loss  0.0549128646585
    Epoch  38 Avg Loss  0.0639970949977
    Epoch  39 Avg Loss  0.0847822522392
    Epoch  40 Avg Loss  0.071867804773
    Epoch  41 Avg Loss  0.0562929863463
    Epoch  42 Avg Loss  0.0511428513438
    Epoch  43 Avg Loss  0.0658274986655
    Epoch  44 Avg Loss  0.0511060538324
    Epoch  45 Avg Loss  0.0497033544187
    Epoch  46 Avg Loss  0.0444484371189
    Epoch  47 Avg Loss  0.0427381113373
    Epoch  48 Avg Loss  0.0401767922213
    Epoch  49 Avg Loss  0.0384558879745
    Training Finished. Saving test images to: ./runs/1504477611.4924326
    Run time (min) 27.0

LR = 0.001, Batch size = 20.

    Tests Passed
    Done.
    INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
    Model loaded.
    In train nn
    Epoch  0 Avg Loss  16.0515673161
    Epoch  1 Avg Loss  0.718232194583
    Epoch  2 Avg Loss  0.672249098619
    Epoch  3 Avg Loss  0.660405659676
    Epoch  4 Avg Loss  0.650631904602
    Epoch  5 Avg Loss  0.641204742591
    Epoch  6 Avg Loss  0.631363129616
    Epoch  7 Avg Loss  0.620187413692
    Epoch  8 Avg Loss  0.607017159462
    Epoch  9 Avg Loss  0.588106671969
    Epoch  10 Avg Loss  0.561252880096
    Epoch  11 Avg Loss  0.522561468681
    Epoch  12 Avg Loss  0.47483954827
    Epoch  13 Avg Loss  0.409901356697
    Epoch  14 Avg Loss  0.349181586504
    Training Finished. Saving test images to: ./runs/1504465006.8358543